In [11]:
from huggingface_hub import login
login()

Text classification

In [12]:
from transformers import pipeline, AutoTokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
torch.manual_seed(0)
#model_id = "tiiuae/falcon-7b-instruct"
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
#model_id = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [3]:
torch.manual_seed(0)
prompt = """Classify the text into neutral, negative or positive.
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment:
"""

sequences = pipe(
    prompt,
    max_new_tokens=10,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Result: Classify the text into neutral, negative or positive. 
Text: This movie is definitely one of my favorite movies of its kind. The interaction between respectable and morally strong characters is an ode to chivalry and the honor code amongst thieves and policemen.
Sentiment:
Positive


In [23]:
torch.manual_seed(1)
prompt = """Return following entities  ['person name,'date','date range'] from the following text in dictionary format.
Text: i am john,writing this mail on 9-2-2024 and need account statement for last two months.
Named entities:
"""

prompt = """You are an advanced language model that specializes in extracting specific entities from text. Given a piece of text, your task is to extract the following entities:

- **Date**: Any specific date mentioned in the text (e.g., "January 1, 2022").
- **Date Range**: Any range of dates mentioned in the text (e.g., "from January 1, 2022 to January 10, 2022",'last two months').
- **Name of a Person**: Any full name of a person mentioned in the text (e.g., "John Doe").

Provide your answer in a dictionary format with the keys "Date", "Date Range", and "Name of a Person". Each key should map to a list of corresponding entities found in the text. If no entities are found for a specific key, the list should be empty.

Text: i am john,writing this mail on 9-2-2024 and need account statement for since 2009.

Here is the text:

"""

sequences = pipe(
    prompt,
    max_new_tokens=100,
    return_full_text = False,
)
print(sequences)
"""for seq in sequences:
    print(f"{seq['generated_text']}")"""

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '```python\ntext = "i am john, writing this mail on 9-2-2024 and need account statement for last since 2009."\n\nentities = {\n    "Date": ["9-2-2024"],\n    "Date Range": ["since 2009"],\n    "Name of a Person": ["john"]\n}\n```\n\nOutput:\n\n```python\n{\''}]


'for seq in sequences:\n    print(f"{seq[\'generated_text\']}")'

In [ ]:

prompt = """Text: The first human went into space and orbited the Earth on April 12, 1961.
Date: 04/12/1961
Text: The first-ever televised presidential debate in the United States took place on September 28, 1960, between presidential candidates John F. Kennedy and Richard Nixon.
Date:"""



As an assistant, your aim is to enhance text and table readability based on specific guidelines:

Reframe sentences and sections for better comprehension.

Eliminate unclear text, e.g., content with excessive symbols or gibberish.

Shorten text without losing information. Suggestion: Summarize lengthy phrases where possible.

Rectify poorly formatted tables, e.g., adjust column alignment for clarity.

Preserve clear, understandable text as is. Example: "Use direct and easily comprehensible sentences."

Refrain from responding if text is entirely unclear or ambiguous, e.g., incomprehensible or garbled content.

Remove standalone numbers or letters not associated with text, e.g., isolated digits or letters lacking context.

Exclude non-factual elements like selection marks, addresses, or drawings.

Ensure modifications maintain the original text's clarity and information conveyed.

Answer back the revised text without additional comments before or after, avoiding comments about how or which guidelines have been followed.


In [2]:

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m